In [1]:
import time
from typing import Dict, Set
from Bio import Entrez, Medline
from naivebayes import NaiveBayes

In [2]:
with open( 'positive_examples.txt' ) as file :
    posids = set( file.read().split( ',' ) )

In [3]:
with open( 'negative_examples.txt' ) as file :
    negids = set( file.read().split( ',' ) )

In [4]:
Class, Document = str, str
data : Dict[ Class, Set[ Document ] ] = { 'positive' : set(), 'negative' : set() }

In [5]:
# getting data
for pmid in posids :
    data[ 'positive' ].add(
        Medline.read(
            Entrez.efetch(
                db = 'pubmed',
                id = pmid,
                email = 'chiodini.zachary@epa.gov',
                retmode = 'text',
                rettype = 'medline'
                )
            ).get( 'AB' )
        )
    time.sleep( 1/3 ) # avoid PubMed ban
for pmid in negids :
    data[ 'negative' ].add(
        Medline.read(
            Entrez.efetch(
                db = 'pubmed',
                id = pmid,
                email = 'chiodini.zachary@epa.gov',
                retmode = 'text',
                rettype = 'medline'
                )
            ).get( 'AB' )
        )
    time.sleep( 1/3 ) # avoid PubMed ban

In [14]:
model = NaiveBayes()
model.trainAndTest( data, iters = 10000 ) # Monte Carlo cross-validation

Examples Trained: 503715
Examples Tested : 496285
Total Examples  : 1000000


In [15]:
import pandas as pd
pd.DataFrame( model.predictions ).fillna( '' )

,positive,negative,model
positive,139485,25997,
negative,11390,319413,
precision,0.842901,0.965569,
truth rate,0.924507,0.924736,
false rate,0.172308,0.0329753,
accuracy,,,0.94(0.06)
